# Counterfactuals Training Data Extraction Experiment

In [1]:
import pandas as pd
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import random
import logging
import warnings
import dice_ml

In [2]:
threads = 31

In [3]:
%run experiment_setup.ipynb

INFO:xai-privacy:Loading dataset 1: heart disease (numeric features) ...
INFO:xai-privacy:Loading dataset 2: census income (categorical features) ...


Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 271 of 1097
Dropped 273 of 1097
Dropped 277 of 1097
Dropped: 2399 of 32561
census: Dropped 3848 of 30162
num: Dropped 19859 of 30162
cat: Dropped 12136 of 30162


In [4]:
logger = logging.getLogger('xai-privacy')

This notebook will test whether training data extraction is possible with counterfactuals (CF) that are drawn from the training data. Training data extraction means an attacker can find out the feature values of samples from the training data without prior knowledge of them. The attacker only has access to the model's prediction function and the explanation.

This attack should be trivial because any counterfactual that is shown as an explanation was picked directly from the training data.

The idea for counterfacutal training data extraction is as follows: The attacker makes repeated queries to the model with random input values. In order to do this, the attacker knows the maximum and minimum value of each feature in the training data (or the categorical values of each feature). The returned counterfactuals are the extracted training data.

First, we implement the `train_explainer` and `training_data_extraction_model_access` functions:

In [5]:
class CounterfactualTDE(TrainingDataExtraction):
    def train_explainer(self, data_train, model):
        # train explainer on training data
        d = dice_ml.Data(dataframe=data_train, continuous_features=self.numeric_features,\
                         outcome_name=self.outcome_name)
        m = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')
        
        # use method "kd-tree" to get counterfactuals drawn from the training data
        return dice_ml.Dice(d, m, method="kdtree")
        
    @staticmethod
    def training_data_extraction_no_model_access(explainer, num_queries, feature_formats, rng):
        rng = np.random.default_rng(rng)
        seed = rng.integers(100000).item()
        random.seed(seed)
        
        # Get all feature names
        feature_names = []
        
        for feature in feature_formats:
            feature_names.append(feature['name'])
        
        samples_df = pd.DataFrame(columns=feature_names)
    
        # This is the default number of counterfactuals per query used on the github page of DiCE
        cfs_per_query = 4
    
        # Generate random samples as queries for the explainer.
        for i in range(num_queries):
            sample = {}
            for feature in feature_formats:
                if feature['isCont']:
                    sample[feature['name']] = rng.integers(feature['min'], feature['max'])
                else:
                    sample[feature['name']] = random.choice(feature['categories'])
            sample_df = pd.DataFrame(sample, index=[0])
            samples_df = pd.concat([samples_df, sample_df], ignore_index=True)

        # Cast categorical features to string again because of DiCE peculiarities
        for feature in feature_formats:
            if not feature['isCont']:
                samples_df[feature['name']] = samples_df[feature['name']].astype(str)
            else:
                samples_df[feature['name']] = samples_df[feature['name']].astype(int)
                
        # Generate counterfactuals for all random query samples
        e1 = explainer.generate_counterfactuals(samples_df, total_CFs=cfs_per_query, desired_class='opposite')
                
        # Collect all extracted samples in this dataframe
        extracted_samples_df = pd.DataFrame(columns=feature_names)
        for index in range(len(samples_df)):
            cfs_of_sample = e1.cf_examples_list[index].final_cfs_df
            logger.debug(f'Sample {index}: Counterfactuals \n {cfs_of_sample.to_numpy()}')

            extracted_samples_df = pd.concat([extracted_samples_df, cfs_of_sample], ignore_index=True)
        
        return extracted_samples_df

# Executing Training Data Extraction

We now generate five counterfactuals for the first sample from the training data to demonstrate counterfactual explanations in general.

In [6]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
d = dice_ml.Data(dataframe=data_heart, continuous_features=numeric_features_heart, outcome_name=outcome_name_heart)

m = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')
# Generating counterfactuals from training data (kd-tree)
exp = dice_ml.Dice(d, m, method="kdtree")

In [7]:
e1 = exp.generate_counterfactuals(features[0:1], total_CFs=5, desired_class="opposite")
e1.visualize_as_dataframe(display_sparse_df=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.34s/it]

Query instance (original outcome : 0)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal,HeartDisease
0,47.0,1.0,1.0,110.0,249.0,0.0,0.0,150.0,0.0,0.0,4.0,4.0,8.0,0.0



Diverse Counterfactual set without sparsity correction (new outcome:  1


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal
388,35.0,1.0,2.0,110.0,257.0,0.0,0.0,140.0,0.0,0.0,4.0,4.0,8.0
434,47.0,1.0,3.0,108.0,243.0,0.0,0.0,152.0,0.0,0.0,1.0,0.0,3.0
705,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,4.0,4.0,7.0
320,46.0,1.0,4.0,120.0,249.0,0.0,2.0,144.0,0.0,0.8,1.0,0.0,7.0
1111,33.0,0.0,4.0,100.0,246.0,0.0,0.0,150.0,1.0,1.0,2.0,4.0,8.0


We can see that the counterfactuals are similar to the query sample and that they have a flipped prediction. These are the two general properties of counterfactual explanations.

We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [8]:
logger.setLevel(logging.DEBUG)
logging.root.setLevel(logging.ERROR)

EXP = CounterfactualTDE(data_heart, numeric_features_heart, outcome_name_heart, random_state=0)
EXP.training_data_extraction_experiment(num_queries=12, model=es.RandomForestClassifier(random_state=0), model_access=False)

logger.setLevel(logging.INFO)

DEBUG:xai-privacy:Numeric Features: ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
DEBUG:xai-privacy:Categorical Features: ['CA', 'ChestPainType', 'ExerciseAngina', 'FastingBS', 'RestingECG', 'ST_Slope', 'Sex', 'Thal']
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:29<00:00,  2.62s/it]
DEBUG:xai-privacy:Sample 0: Counterfactuals 
 [[51.0 '1.0' '3.0' 110.0 175.0 '0.0' '0.0' 123.0 '0.0' 0.6 '1.0' '0.0'
  '3.0']
 [49.0 '1.0' '3.0' 118.0 149.0 '0.0' '2.0' 126.0 '0.0' 0.8 '1.0' '3.0'
  '3.0']
 [55.0 '1.0' '4.0' 116.0 186.0 '1.0' '1.0' 102.0 '0.0' 0.0 '4.0' '4.0'
  '8.0']
 [54.0 '1.0' '4.0' 120.0 188.0 '0.0' '0.0' 113.0 '0.0' 1.4 '2.0' '1.0'
  '7.0']]
DEBUG:xai-privacy:Sample 1: Counterfactuals 
 [[55.0 '1.0' '3.0' 0.0 0.0 '0.0' '0.0' 155.0 '0.0' 1.5 '2.0' '4.0' '8.0']
 [32.0 '1.0' '1.0' 95.0 0.0 '2.0' '0.0' 127.0 '

Total time: 30.86s (training model: 0.38s, training explainer: 0.06s, experiment: 30.42s)
Number of extracted samples: 39
Number of accurate extracted samples: 39
Precision: 1.0, recall: 3.25


The proof of concept should show that each extracted sample is an actual training sample (precision of 100%). Recall is above 100% because this method can extract multiple samples per query (multiple counterfactuals are returned). Recall will reach a reasonable value if the experiment is executed for the full training data. In this case, the attack cannot return more samples than the number of queries because the attack is limited by the number of training samples.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [9]:
results_ = {'dataset': [], 'model': [], 'precision': [], 'recall': []}

results = pd.DataFrame(data = results_)

In [10]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

# We set the number of extractions to the length of the dataset
num_queries_dict = { 'heart': len(data_heart), 'heart numeric': len(data_heart_num), 'heart categorical': len(data_heart_cat), 'census': len(data_census), 'census numeric': len(data_census_num), 'census categorical': len(data_census_cat)}

In [11]:
# remove pandas warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [12]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(CounterfactualTDE, dataset_dicts, model_dicts, random_state=0, num_queries=num_queries_dict, model_access=False, threads=threads, results_table=results, is_mem_inf=False, convert_cat_to_str=True)

dataset: heart, model: decision tree


100%|██████████| 27/27 [00:57<00:00,  1.46s/it]


Total time: 59.68s (training model: 0.04s, training explainer: 0.08s, experiment: 59.56s)
Number of extracted samples: 540
Number of accurate extracted samples: 540
Precision: 1.0, recall: 0.6537530266343826
dataset: heart, model: random forest


100%|██████████| 27/27 [02:01<00:00,  4.85s/it]


Total time: 125.34s (training model: 0.78s, training explainer: 0.09s, experiment: 124.47s)
Number of extracted samples: 482
Number of accurate extracted samples: 482
Precision: 1.0, recall: 0.5835351089588378
dataset: heart, model: neural network


...

Total time: 745.76s (training model: 1.23s, training explainer: 0.56s, experiment: 743.97s)
Number of extracted samples: 14178
Number of accurate extracted samples: 14178
Precision: 1.0, recall: 0.7865305669588373
dataset: census categorical, model: random forest


100%|██████████| 581/581 [21:28<00:00,  1.65s/it]


Total time: 1372.50s (training model: 16.91s, training explainer: 0.48s, experiment: 1355.11s)
Number of extracted samples: 14038
Number of accurate extracted samples: 14038
Precision: 1.0, recall: 0.7787640075446577
dataset: census categorical, model: neural network


/home/i40/langema/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 582/582 [22:44<00:00,  1.18it/s]


Total time: 1525.03s (training model: 88.70s, training explainer: 0.41s, experiment: 1435.93s)
Number of extracted samples: 12541
Number of accurate extracted samples: 12541
Precision: 1.0, recall: 0.6957172972373239


# Results

Precision is the percentage of extracted samples that is actually from the training data. 

Recall is the ratio of the number extracted training samples to all training samples.

In [13]:
results

,dataset,model,precision,recall
0,heart,decision tree,1.0,0.653753
1,heart,random forest,1.0,0.583535
2,heart,neural network,1.0,0.567797
3,heart numeric,decision tree,1.0,0.645631
4,heart numeric,random forest,1.0,0.611650
5,heart numeric,neural network,1.0,0.598301
6,heart categorical,decision tree,1.0,0.864634
7,heart categorical,random forest,1.0,0.790244
8,heart categorical,neural network,1.0,0.786585
9,census,decision tree,1.0,0.077031


In [14]:
results.to_csv('results/2-1-cf-training-data-extraction-results.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

In our experiments, training data extraction with counterfactuals drawn from the training data has a recall between 45% and 67% for numeric data and 30% to 64% for categorical data. Since the attack cannot produce any false positive samples, precision is always 100%.